# Implementar un MLP con PyTorch para clasificación basado en el dataset de agresividad

1. **Definir los preprocesamientos para el texto**:  
   - convertir a minúsculas
   - normalizar el texto: borrar símbolos, puntuación, caracteres duplicados, etc.

2. **Separar los datos para entrenamiento y prueba**:  
   - Crear los dataset de entrenamiento y test con al función train_test_split 

3. **Construir la matriz de Documento-Término**:  
   - Definir los parámetros para usar unigramas
   - Usar la clase TfidfVectorizer para construir la matriz con los datos de entrenamiento
   
4. **Preparar los lotes de datos (minibatches) para el entrenamiento de la red**:  
   - Definir los minibatches con la matriz TFIDF construida

5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   


# Definición de los datos y minibatches

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split


In [ ]:
# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset = pd.read_json("./data/data_aggressiveness_es.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset['klass'].to_numpy()



# TODO: Definir las funciones de preprocesamiento de texto vinculadas al proceso de creación de la matriz 
# Documeno-Término creada con TfidfVectorizer.

_STOPWORDS = stopwords.words("spanish")  # agregar más palabras a esta lista si es necesario

# Normalización del texto

import unicodedata
import re
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto


# Preprocesamiento personalizado 
def mi_preprocesamiento(texto):
    #convierte a minúsculas el texto antes de normalizar
    tokens = word_tokenize(texto.lower())
    texto = " ".join(tokens)
    texto = normaliza_texto(texto)
    return texto
    
# Tokenizador personalizado 
def mi_tokenizador(texto):
    # Elimina stopwords: palabras que no se consideran de contenido y que no agregan valor semántico al texto
    #print("antes: ", texto)
    texto = [t for t in texto.split() if t not in _STOPWORDS]
    #print("después:",texto)
    return texto

# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))

# TODO: Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)


X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

# Divide el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)



# TODO: Crear la matriz Documento-Término con el dataset de entrenamiento: tfidfVectorizer


vec_tfidf = TfidfVectorizer(analyzer="word", preprocessor=mi_preprocesamiento, tokenizer=mi_tokenizador,  ngram_range=(1,1))
X_train_tfidf = vec_tfidf.fit_transform(X_train)

NUM_CLASSES = 2

# Codificación de la salida onehot

Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()
Y_test_one_hot = nn.functional.one_hot(torch.from_numpy(Y_test), num_classes=NUM_CLASSES).float()
Y_val_one_hot = nn.functional.one_hot(torch.from_numpy(Y_val), num_classes=NUM_CLASSES).float()


# Convertir a matriz densa de tipo de dato float32 (tipo de dato por default en Pytorch)
X_train_tfidf = X_train_tfidf.toarray().astype(np.float32)

# Tranforma los datos de validación al espacio de representación del entrenamiento
X_val_tfidf = vec_tfidf.transform(X_val)

# Convertir a matriz densa de tipo de dato float32 (tipo de dato por default en Pytorch)
X_val_tfidf = X_val_tfidf.toarray().astype(np.float32)



# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [ ]:
X_train.shape, Y_train.shape

In [ ]:
Y_train_one_hot.shape, Y_test_one_hot.shape,  Y_val_one_hot.shape

# Definición de la arquitectura de la red

In [ ]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        input_size_h1 = 128
        input_size_h2 = 8 
        self.fc1 = nn.Linear(input_size, input_size_h1)
        # PReLU tiene parámetros aprendibles: Se recomienda una función de activación independiente por capa
        self.act1= nn.PReLU()

        self.fc2 = nn.Linear(input_size_h1, input_size_h2)
        # PReLU tiene parámetros aprendibles: Se recomienda una función de activación independiente por capa
        self.act2= nn.PReLU()

        self.output = nn.Linear(input_size_h2, output_size)
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.output.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        
        if self.output.bias is not None:
            nn.init.zeros_(self.output.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.output(x)
        # Nota la última capa de salida 'output' no se activa debido a que CrossEntropyLoss usa LogSoftmax internamente. 
        return x

# Entrenamiento de la red

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
input_size =  X_train_tfidf.shape[1]

output_size = 2   # 2 clases

epochs = 10 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.01 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 128 # definir el tamaño del lote de procesamiento 


# TODO: Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_train_t = torch.from_numpy(X_train_tfidf)
Y_train_t = Y_train_one_hot

X_val_t = torch.from_numpy(X_val_tfidf)

# Crear la red
model = MLP(input_size, output_size)

# Definir la función de pérdida
# Mean Square Error (MSE)
# criterion = nn.MSELoss()
# criterion = nn.BCELoss() 
criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Stochastic Gradient Descent (SGD)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")


for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    model.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  W = W - learning_rate * dE/dW
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    model.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = model(X_val_t)
        # Aplica softmax para obtener las probabilidades en la evaluación
        y_pred = torch.softmax(y_pred, dim=1)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)        
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))




### Modo para predicción de datos

In [ ]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_test_tfidf = vec_tfidf.transform(X_test)

# Convertir a matriz densa de tipo de dato float32 (tipo de dato por default en Pytorch)
X_test_tfidf = X_test_tfidf.toarray().astype(np.float32)
X_t = torch.from_numpy(X_test_tfidf)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
model.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= model(X_t)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(y_pred_test)


### Evaluación

In [ ]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


print(confusion_matrix(Y_test, y_pred_test))
print(classification_report(Y_test, y_pred_test, digits=4, zero_division='warn'))


### Evaluación de datos nuevos

In [ ]:

x_new_data = ["Ese perro me robo mis cosas", "ese hdp se llevo el dinero", "mi app de calendario no sirve"]
x_new_data_tfidf = vec_tfidf.transform(x_new_data)
# Convertir a matriz densa de tipo de dato float32 (tipo de dato por default en Pytorch)
x_new_data_tfidf = x_new_data_tfidf.toarray().astype(np.float32)
X_new_t = torch.from_numpy(x_new_data_tfidf)


model.eval()  # Establecer el modo del modelo a "evaluación"
with torch.no_grad():  # No  calcular gradientes 
    y_pred = model(X_new_t)
    y_pred = torch.argmax(y_pred, dim=1)
    print(le.inverse_transform(y_pred))

# Ejercicio.  Modificar la red neuronal MLP con las siguientes características:
- ## Arquitectura:
    - ### 4 capas ocultas
    - ### 2 salidas
    - ### Funciones de activación en capas ocultas ELU
    - ### Número de neuronas por capa oculta a su consideración
- ## Prepocesamiento:
    - ### Normalización
    - ### Repesentación de características: unigramas sin STOPWORDS y con stemming
    - ### Pesado TF-IDF

- ## Evaluación del rendimiento del modelo: 
    - ### 1. Paticiones train (80%), test (20%), validación (10% del train)
    - ### 2. Validación cruzada k-folds = 5


In [18]:
import unicodedata
import re
import torch
import numpy as np
import pandas as pd
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

_STOPWORDS = stopwords.words("spanish")
RANDOM_STATATE = 42
torch.manual_seed(RANDOM_STATATE)
np.random.seed(RANDOM_STATATE)

PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto

def mi_tokenizador(texto):
    stemmer = SnowballStemmer("spanish")
    texto = [stemmer.stem(t) for t in texto.split() if t not in _STOPWORDS]
    return texto

def create_minibatches(X, Y, batch_size):
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

In [19]:
class MLPELU4L(nn.Module):
    def __init__(self, sizes):
        super().__init__()
        self.fcl = nn.ModuleList()
        self.act = []
        for i in range(len(sizes) - 2):
            self.fcl.append(nn.Linear(sizes[i], sizes[i + 1]))
        for i in range(len(sizes) - 2):
            nn.init.xavier_uniform_(self.fcl[i].weight)
            nn.init.zeros_(self.fcl[i].bias)
            self.act.append(nn.PReLU())
    
    def forward(self, X):
        x = X
        for i in range(len(self.fcl)):
            x = self.fcl[i](x)
            x = self.act[i](x)
        return x

In [20]:
def vectorizar(X, Y, tipo_vec='TF-IDF', test_size=0.2, val_size=0.1, ngram_range=(1,1)):
    vec = TfidfVectorizer(analyzer="word", preprocessor=normaliza_texto, tokenizer=mi_tokenizador, ngram_range=ngram_range)
    le = LabelEncoder()
    
    Y_vec = le.fit_transform(Y)
    X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_vec, test_size=test_size, stratify=Y_vec, random_state=RANDOM_STATATE)
    X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=val_size, stratify=Y_train, random_state=RANDOM_STATATE)
    
    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)
    X_val_vec = vec.transform(X_val)
    
    return X_train_vec, X_test_vec, X_val_vec, Y_train, Y_test, Y_val

def entrenar(X, Y, epocas=10, learning_rate=0.2, batch_size=128):
    X_train, X_test, X_val, Y_train, Y_test, Y_val = vectorizar(X, Y)
    
    X_train_np = X_train.toarray().astype(np.float32)
    X_train_torch = torch.from_numpy(X_train_np)
    X_test_np = X_test.toarray().astype(np.float32)
    X_test_torch = torch.from_numpy(X_train_np)
    X_val_np = X_val.toarray().astype(np.float32)
    X_val_torch = torch.from_numpy(X_val_np)
    
    Y_train = torch.from_numpy(Y_train)
    Y_test = torch.from_numpy(Y_test)
    Y_val = torch.from_numpy(Y_val)
    
    modelo = MLPELU4L(sizes=[X_train_torch.shape[1], 512, 256, 128, 64, 2])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(modelo.parameters(), lr=learning_rate)
    

    
    print("Iniciando entrenamiento en PyTorch")
    for e in range(epocas):
        modelo.train()
        lossTotal = 0   
        dataloader = create_minibatches(X_train_torch, Y_train, batch_size=batch_size)
        for X_tr, Y_tr in dataloader:
            optimizer.zero_grad()
            Y_pred = modelo(X_tr)
            loss = criterion(Y_pred, Y_tr)
            lossTotal += loss.item()
            loss.backward()
            optimizer.step()
            if np.random.random() < 0.1:
                print(f"Batch Error : {loss.item()}")
        print(f"Época {e+1}/{epocas}, Pérdida: {lossTotal/len(dataloader)}")

        modelo.eval()
        with torch.no_grad():
            Y_pred = modelo(X_val_torch)
            Y_pred = torch.softmax(Y_pred, dim=1)
            Y_pred = torch.argmax(Y_pred, dim=1)
            print(f"Época {e+1}/{epocas}")    
            print("P=", precision_score(Y_val, Y_pred, average='macro'))
            print("R=", recall_score(Y_val, Y_pred, average='macro'))
            print("F1=", f1_score(Y_val, Y_pred, average='macro'))
            print("Acc=", accuracy_score(Y_val, Y_pred))
        

In [21]:
dataset = pd.read_json("./data/data_aggressiveness_es.json", lines=True)
X = dataset['text'].to_numpy()
Y = dataset['klass'].to_numpy()

entrenar(X, Y)

/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Iniciando entrenamiento en PyTorch
Batch Error : 573.0330810546875
Batch Error : 525.5831298828125
Época 1/10, Pérdida: 729.5179196883893
Época 1/10
P= 0.314819513320263
R= 0.22469485740730027
F1= 0.22225891817508203
Acc= 0.732360097323601
Batch Error : 1603.833740234375
Batch Error : 278.5577087402344
Batch Error : 70.94841766357422


/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Batch Error : 289.41912841796875
Batch Error : 144.9371795654297
Época 2/10, Pérdida: 327.2862864527209
Época 2/10
P= 0.49331616384596516
R= 0.48387998688802764
F1= 0.48813366054386353
Acc= 0.7858880778588808
Batch Error : 230.195556640625


/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


...

/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Batch Error : 661.9522094726562
Época 5/10, Pérdida: 606.901622903758
Época 5/10
P= 0.7453161592505855
R= 0.6514866662810204
F1= 0.6679062008566687
Acc= 0.7737226277372263
Batch Error : 171.99435424804688
Batch Error : 980.0784301757812
Batch Error : 1880.0615234375
Batch Error : 5861.51025390625
Batch Error : 337.2598876953125
Época 6/10, Pérdida: 1581.9422018116918
Época 6/10
P= 0.6281812125249834
R= 0.6112975067970151
F1= 0.6166192557644719
Acc= 0.7055961070559611
Batch Error : 1506.0289306640625
Batch Error : 3650.49072265625
Época 7/10, Pérdida: 2234.8515119881467
Época 7/10
P= 0.33562191223481547
R= 0.35111644588419044
F1= 0.3321154828819622
Acc= 0.683698296836983
Batch Error : 206.99044799804688
Batch Error : 1801.4664306640625


/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Batch Error : 2987.9091796875
Época 8/10, Pérdida: 9083.675990663725
Época 8/10
P= 0.6877944976440189
R= 0.7144096720078672
F1= 0.6944769992164531
Acc= 0.7299270072992701
Batch Error : 2059.230712890625
Batch Error : 2273.81494140625
Batch Error : 7876.955078125
Época 9/10, Pérdida: 13183.595919905038
Época 9/10
P= 0.7366640789619607
R= 0.7468907271359981
F1= 0.7412442966788102
Acc= 0.7834549878345499
Batch Error : 18902.416015625
Batch Error : 3540.15234375
Época 10/10, Pérdida: 7325.952038995151
Época 10/10
P= 0.49150984516838175
R= 0.49341509033763326
F1= 0.4923306984666174
Acc= 0.781021897810219


/home/luis-beto/anaconda3/envs/RNA/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
